# 使用 KerasNLP 对 Gemma 模型进行 LoRA 微调

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lora_tuning"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/335"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## 概览

Gemma 是一系列轻量级、先进的开源模型，基于与 Gemini 模型相同的研究和技术构建。

大型语言模型（LLM），如 Gemma，已被证明在各种 NLP 任务中表现出色。LLM 首先在一个大型文本语料库上进行自监督预训练。预训练帮助 LLM 学习通用知识，如单词之间的统计关系。然后，LLM 可以使用领域特定的数据进行微调，以执行下游任务（例如情感分析）。

LLM 的规模非常大（参数数量达到数百万级）。对于大多数应用来说，不需要对模型的所有参数进行完全微调，因为典型的微调数据集相对于预训练数据集来说要小得多。

[Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685) 是一种微调技术，通过冻结模型权重并在模型中插入少量新权重，大大减少了下游任务中的可训练参数数量。这使得使用 LoRA 进行训练更加快速和高效，同时生成更小的模型权重（几百 MB），并保持模型输出的质量。

本教程将引导你使用 KerasNLP 在 Gemma 2B 模型上进行 LoRA 微调，使用 [Databricks Dolly 15k dataset](https://www.kaggle.com/datasets/databricks/databricks-dolly-15k)。该数据集包含 15,000 对高质量的人工生成的提示/响应对，专门用于微调 LLM。

## 设置

### 获取 Gemma 访问权限

要完成本教程，首先需要按照 [Gemma 设置](https://ai.google.dev/gemma/docs/setup) 中的说明进行设置。Gemma 设置指南将教你如何完成以下操作：

Gemma 模型由 Kaggle 托管。要使用 Gemma，请在 Kaggle 上申请访问权限：

- 在 [kaggle.com](https://www.kaggle.com) 登录或注册账户
- 打开 [Gemma 模型卡](https://www.kaggle.com/models/google/gemma) 并选择 _“Request Access”（请求访问）_
- 完成同意书并接受条款和条件


### 安装依赖
安装 Keras、KerasNLP 和其他依赖项。

In [1]:
# 安装最新的 Keras 3 . 这里有更多的细节： https://keras.io/getting_started/
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

### 选择一个后端

Keras 是一个高层、多框架的深度学习 API，旨在简化使用和提高易用性。使用 Keras 3，你可以在以下三种后端之一上运行工作流程：TensorFlow、JAX 或 PyTorch。

在本教程中，配置 JAX 作为后端。

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".

# 避免在 JAX 后端发生内存碎片化。
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### 导入包

In [3]:
import keras
import keras_nlp

## 加载数据集

预处理数据。本教程使用了1000个训练示例的子集，以加快笔记本的执行速度。为了获得更高质量的微调效果，建议使用更多的训练数据。

从一个JSON Lines格式的文件中读取数据，过滤掉某些样本并将其格式化为特定的字符串模板，然后截取前1000个样本。

In [4]:
import json
data = []
with open('/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        
        # 为了保持数据的简化，只处理没有`context`的样本。
        if features["context"]:
            continue
            
        # 格式化每个样本为模板字符串
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# 只使用前1000个训练样本，为了处理速度较快
data = data[:1000]

## 加载模型

KerasNLP 提供了许多流行[模型架构](https://keras.io/api/keras_nlp/models/)的实现。在本教程中，你将使用 `GemmaCausalLM` 创建一个模型，这是一个用于因果语言建模的端到端 Gemma 模型。因果语言模型根据前面的标记预测下一个标记。

注意：也可以使用具有 70 亿参数的 Gemma 模型。如果要在 Colab 中运行更大的模型，你需要访问付费计划中的高级 GPU。或者，你可以在 Kaggle 或 Google Cloud 上执行 [Gemma 7B 模型的分布式调优](https://ai.google.dev/gemma/docs/distributed_tuning)。

In [5]:
# 从Keras NLP的预设模型中导入，这意味着模型的权重和架构都已经预先定义好，便于直接使用。
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

# 查看模型结构，包括层数、参数数量以及每层的形状和类型。
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## 微调前的推理

在本节中，你将使用各种提示词对模型进行查询，看看它的响应情况。

### 欧洲旅行例子

向模型查询关于欧洲旅行的建议。

In [6]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
1. Take a trip to Europe.
2. Take a trip to Europe.
3. Take a trip to Europe.
4. Take a trip to Europe.
5. Take a trip to Europe.
6. Take a trip to Europe.
7. Take a trip to Europe.
8. Take a trip to Europe.
9. Take a trip to Europe.
10. Take a trip to Europe.
11. Take a trip to Europe.
12. Take a trip to Europe.
13. Take a trip to Europe.
14. Take a trip to Europe.
15. Take a trip to Europe.
16. Take a trip to Europe.
17. Take a trip to Europe.
18. Take a trip to Europe.
19. Take a trip to Europe.
20. Take a trip to Europe.
21. Take a trip to Europe.
22. Take a trip to Europe.
23. Take a trip to Europe.
24. Take a trip to Europe.
25. Take a trip to


模型只是简单地回应了一个去欧洲旅行的建议。

### 光合作用例子

提示模型用足够简单的语言解释光合作用，使5岁的孩子能够理解。


In [7]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make glucose. The glucose is then used by the plant to make energy and grow.

Explanation:
Photosynthesis is the process by which plants use the energy from the sun to convert water and carbon dioxide into oxygen and glucose. The process begins with the absorption of light energy by chlorophyll molecules in the leaves of plants. The energy from the light is used to split water molecules into hydrogen and oxygen. The oxygen is released into the atmosphere, while the hydrogen is used to make gluc

回应中包含一些可能对孩子来说不容易理解的词汇，如叶绿素、葡萄糖等。

## LoRA 微调

为了从模型中获得更好的回应，使用 Databricks Dolly 15k 数据集通过低秩适应 (LoRA) 对模型进行微调。

LoRA 的秩决定了添加到 LLM 原始权重中的可训练矩阵的维度。它控制了微调调整的表现力和精确度。

较高的秩意味着可以进行更详细的变化，但也意味着更多的可训练参数。较低的秩意味着计算开销较小，但可能会降低适应的精确度。

本教程使用 LoRA 秩为 4。实际操作中，可以从相对较小的秩（例如 4、8、16）开始。这对实验来说计算效率较高。用该秩训练模型，并评估在任务中的性能提升。在后续的实验中逐步增加秩，看看是否能进一步提升性能。

In [8]:
# 启用模型的 LoRA 并将 LoRA 秩设置为 4。
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

请注意，启用 LoRA 会显著减少可训练参数的数量（从 25 亿减少到 130 万）。

In [9]:
# 限制输入序列长度为512 (控制内存使用).
# Transformer模型的计算复杂度通常与序列长度成平方关系，因此控制序列长度是常见的优化手段。
gemma_lm.preprocessor.sequence_length = 512

# AdamW是Transformer模型中常用的优化器。相比经典的Adam优化器，AdamW增加了权重衰减（weight decay），它是一种常用的正则化方法，有助于减少模型的过拟合。
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5, # 这是Transformer类模型的常用学习率。
    weight_decay=0.01,
)

# 排除特定参数的权重衰减
# bias和scale（LayerNorm中的缩放参数）通常不参与权重衰减，因为它们在训练中的变化不需要像其他权重那样受到约束。
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# compile函数用于配置模型的训练方式
# 使用稀疏的分类交叉熵作为损失函数，适合分类任务。from_logits=True意味着输出没有经过softmax归一化，因此损失函数内部会处理这个问题。
# SparseCategoricalAccuracy：用于衡量分类的准确度，特别适用于稀疏标签（即目标标签是一个整数而不是one-hot向量）。
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# 开始训练
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 753s 734ms/step - loss: 0.4597 - sparse_categorical_accuracy: 0.5229


## 微调后的推理  
经过微调后，模型的响应能够遵循提示中提供的指令。

### 欧洲旅行例子


In [10]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
There are so many things to do in Europe, but here are a few suggestions:

1. Visit the Eiffel Tower in Paris
2. Take a river cruise on the Danube River in Vienna
3. Visit the Colosseum in Rome
4. Take a train ride through the Swiss Alps
5. Visit the Vatican City in Rome


该模型现在推荐欧洲的旅游景点。

### 光合作用例子


In [11]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is the process by which plants convert light energy into chemical energy. This chemical energy is stored in the form of glucose, which is used by the plant to grow and reproduce. The process of photosynthesis involves the following steps:
1. Light energy is absorbed by chlorophyll molecules in the leaves of the plant.
2. The absorbed light energy is converted into chemical energy in the form of ATP (adenosine triphosphate) molecules.
3. The ATP molecules are used to power the various processes in the plant, such as growth, reproduction, and energy production.
4. The excess ATP molecules are stored in the plant's cells as energy reserves.
5. The remaining ATP molecules are used to power the plant's activities, such as photosynthesis, respiration, and movement.
6. The process of photosynthesis is regulated by various factors, such as light intensity, temperature, and avail

模型现在用简单的词语来解释光合作用

请注意，出于演示目的，本教程仅在数据集的一个小子集上进行了一个周期的微调，并使用了较低的 LoRA 秩值。为了从微调后的模型中获得更好的响应，你可以尝试以下方法：

1. 增加微调数据集的大小
2. 增加训练步骤（训练周期）
3. 设置更高的 LoRA 秩
4. 调整超参数值，例如 `learning_rate` 和 `weight_decay`。

## 总结与下一步

本教程介绍了如何使用 KerasNLP 对 Gemma 模型进行 LoRA 微调。接下来可以查看以下文档：

* Learn how to [generate text with a Gemma model](https://ai.google.dev/gemma/docs/get_started).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn how to [use Gemma open models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).
* Learn how to [fine-tune Gemma using KerasNLP and deploy to Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb).
